# Объединение датафреймов

In [1]:
import pandas as pd

## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [70]:
#ratings.loc[ratings.userId == 15]

In [3]:
aggregations = {
    'timestamp': ['min', 'max'],    
    'rating': 'count'
}
ratings = ratings.groupby('userId').agg(aggregations)

In [4]:
ratings_over100 = ratings[ratings['rating']['count'] > 100].reset_index()
ratings_over100

userId   timestamp             rating
                   min         max  count
0        4   949778714   949982274    204
1        8  1154389340  1154474527    116
2       15   997937239  1469330735   1700
3       17  1127468587  1127476640    363
4       19   855190091   855195373    423
..     ...         ...         ...    ...
253    656   986240991   986244044    128
254    659   834598040   866207451    142
255    664  1343731283  1441911722    519
256    665   992836298  1046967769    434
257    671  1063500751  1074784735    115

[258 rows x 4 columns]

In [5]:
def get_TTL(row):
    return str(int(row['timestamp']['max']) - int(row['timestamp']['min']))    

ratings_over100['TTLmean'] = ratings_over100.apply(get_TTL, axis = 1)
ratings_over100

userId   timestamp             rating    TTLmean
                   min         max  count           
0        4   949778714   949982274    204     203560
1        8  1154389340  1154474527    116      85187
2       15   997937239  1469330735   1700  471393496
3       17  1127468587  1127476640    363       8053
4       19   855190091   855195373    423       5282
..     ...         ...         ...    ...        ...
253    656   986240991   986244044    128       3053
254    659   834598040   866207451    142   31609411
255    664  1343731283  1441911722    519   98180439
256    665   992836298  1046967769    434   54131471
257    671  1063500751  1074784735    115   11283984

[258 rows x 5 columns]

## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
    таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
    аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [7]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)


In [8]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)


In [9]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)


In [10]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)


In [13]:
rzd_grouped = rzd.groupby('client_id').sum()
auto_grouped = auto.groupby('client_id').sum()
air_grouped = air.groupby('client_id').sum()
revenue_merge = pd.merge(rzd_grouped, auto_grouped, on = 'client_id', how = 'outer')
revenue_merge = pd.merge(revenue_merge, air_grouped, on = 'client_id', how = 'outer')
revenue_merge

,rzd_revenue,auto_revenue,air_revenue
client_id,,,
111,1093.0,NaN,NaN
112,2810.0,NaN,NaN
113,10283.0,57483.0,NaN
114,5774.0,83.0,NaN
115,981.0,912.0,81.0
116,NaN,4834.0,4.0
117,NaN,98.0,13.0
118,NaN,NaN,173.0


In [12]:
pd.merge(revenue_merge, client_base, on = 'client_id', how = 'outer')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. 
Как бы вы добавили эти сведения в таблицу визитов и покупок? 
Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [ ]:
Можно взять радиус местоположения пользователей.
((x-x_0)**2 + (y-y_0)**2)**0.5
Получим число, относительно которого можно разделить пользователей на группы по удаленности от местоположения магазина.
Если у одного и того же пользователя координаты сильно удалены друг от друга, то такой пользователь попадет в 2 выборки.
В качестве доп.признака можно получить связанность пользователей. 
Т.е. одинаковые координаты у разных пользователей могут указать на то, что это один человек.

In [ ]:
visits = pd.DataFrame(
    {   'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

In [ ]:
purchases = pd.DataFrame(
    {   'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

In [38]:
location = pd.DataFrame(
    {   'user_id': [11, 22, 55, 11, 77],
        'Latitude': [-34.58234, -15.784657, -33.4575, 4.60775, -34.58234],
        'Longitude': [-58.66432, -47.91674, -70.667655, -74.0875, -58.66432],
    }
)

In [39]:
#для примера возьмем, что местоположение магазина (0, 0)

def radius(row):
        return (float(row['Latitude'])**2 + float(row['Longitude'])**2)**0.5
        
location['radius'] = location.apply(radius, axis = 1)


#location['Latitude_round'] = location['Latitude'].apply(lambda x: round(x, 2))


In [40]:
location

,user_id,Latitude,Longitude,radius
0,11,-34.582340,-58.664320,68.098757
1,22,-15.784657,-47.916740,50.449672
2,55,-33.457500,-70.667655,78.187734
3,11,4.607750,-74.087500,74.230647
4,77,-34.582340,-58.664320,68.098757
